In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
from hmpai.utilities import pad_to_max_sample_length
from hmpai.data import add_stage_dimension
import xarray as xr

### Split data into segments

In [2]:
data_path = Path("../data/sat1/stage_data_100hz.nc")
output_path = Path("../data/sat1/split_stage_data_100hz.nc")
# Takes ~9 minutes
output_data = add_stage_dimension(data_path)
# Loses other data variables

Finding stage changes
Starting segmentation


  0%|          | 0/4521 [00:00<?, ?it/s]

Combining segments


In [4]:
# Save segmented data
output_data.to_netcdf(output_path)

#### Split data using another labelled dataset

In [ ]:
data_path = Path("data/sat1/stage_data.nc")
merge_dataset = xr.load_dataset(Path("data/sat1/data_unprocessed.nc"))
output_data = add_stage_dimension(data_path, merge_dataset)

In [11]:
output_data

<xarray.Dataset>
Dimensions:      (channels: 30, samples: 770, epochs: 200, participant: 25,
                  labels: 5)
Coordinates:
  * channels     (channels) object 'Fp1' 'Fp2' 'AFz' 'F7' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 0 1 2 3 4 5 6 7 ... 763 764 765 766 767 768 769
  * epochs       (epochs) int64 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * participant  (participant) object '0001' '0002' '0003' ... '0024' '0025'
  * labels       (labels) <U13 'confirmation' 'decision' ... 'response'
    stim         (participant, epochs) float64 nan 1.0 1.0 1.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object nan 'resp_left' ... 'resp_left'
    RT           (participant, epochs) float64 nan 0.683 1.068 ... nan 1.02
    cue          (participant, epochs) object nan 'SP' 'AC' ... 'SP' nan 'AC'
    movement     (participant, epochs) object nan 'stim_left' ... 'stim_right'
    trigger      (participant, epochs) object nan ... 'AC/stim_right/resp_left'
Data variables:
    data         (participant, epochs, labels, channels, samples) float64 nan...

In [1]:
output_path = Path("data/sat1/split_stage_data_unprocessed.nc")
output_data.to_netcdf(output_path)

NameError: name 'Path' is not defined

### Deprecated

In [ ]:
# %%time
# 2488 labels/segments
stage_data = xr.load_dataset(data_path)
dimensions = stage_data.labels.shape
segments = []
labels = []
for participant in range(dimensions[0]):
    for epoch in range(dimensions[1]):
        trial = stage_data.isel(participant=participant, epochs=epoch)
        # If trial contains anything other than empty strings, at least one processing stage has been identified and labelled
        if np.any(trial.labels != ""):
            np_labels = trial.labels.to_numpy()
            # Find locations where string changes in a sequence of strings to find changes in stages
            changes = np.where(np_labels[:-1] != np_labels[1:])[0] + 1
            # Get EEG data for participant/epoch

            # Create list of EEG data splits
            splits = np.split(trial.data, changes, 1)
            label_splits = np.split(trial.labels, changes, 0)
            for split, label_split in zip(splits, label_splits):
                if np.any(label_split != ""):
                    # Sanity check
                    if np.any(np.isnan(split)):
                        print(f"NAN FOUND participant {participant}, epoch {epoch}")
                        print(f"Label: {label_split[0].item()}")
                        print(split.to_numpy()[0, :])
                        raise ValueError(
                            "Split found that is fully labeled but still contains NaNs in data"
                        )
                    # Add split and label to lists
                    segments.append(split.data)
                    labels.append(label_split[0].item())
        else:
            print(f"Trial unusable: participant {participant}, epoch {epoch}")

# Pad to max_sample_length, can differ per data set or subset
max_sample_length = max(segment.shape[1] for segment in segments)
segments = np.stack(
    [pad_to_max_sample_length(segment, max_sample_length) for segment in segments]
)